In [148]:
%pip install pandas
%pip install openpyxl
import pandas as pd

data_source = 'local'

prilohy_path = {
    'A': '../data/prilohaA-napocet.xlsx',
    'B': '../data/prilohaB-crepc.xlsx',
    'C': '../data/prilohaC-monografie.xlsx',
    'D': '../data/prilohaD-granty.xlsx',
} if data_source == 'local' else {
    'A': 'https://www.minedu.sk/data/att/25844.xlsx',
    'B': 'https://www.minedu.sk/data/att/25563.xlsx',
    'C': 'https://www.minedu.sk/data/att/25537.xlsx',
    'D': 'https://www.minedu.sk/data/att/25538.xlsx',
}
prilohaA = pd.read_excel(prilohy_path['A'], sheet_name = None, header=None)
print(prilohaA.keys())
# ['E1_alokácia', 'E2_zamestnanci_2021', 'E3_oblasti', 'E4a_M1_prirodne', 'E4b_M2_technicke', 'E4c_M3_lekarske', 'E4d_M4_polno_les_vet', 'E4e_M5_spolocenske', 'E4f_M6_humanitne', 'E4g_M6_umenie']
prilohaB = pd.read_excel(prilohy_path['B'], sheet_name = None)
print(prilohaB.keys())
# ['2021 - 3']
prilohaC = pd.read_excel(prilohy_path['C'], sheet_name = None)
print(prilohaC.keys())
# ['crepc 2020', 'crepc 2021', 'crepc 2022', 'crepc 2022-2', 'CVTI_NORDIC', 'Nordic_CVTI 2020', 'Nordic_CVTI 2021', 'vystupy 2020', 'vystupy 2021', 'Hárok1']
prilohaD = pd.read_excel(prilohy_path['D'], sheet_name = None)
print(prilohaD.keys())
# ['T3 - výsk. zahr. grant. schémy', 'old', 'oblasti výskumu', 'VŠ', 'Odbory VaT']



Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
dict_keys(['E1_alokácia', 'E2_zamestnanci_2021', 'E3_oblasti', 'E4a_M1_prirodne', 'E4b_M2_technicke', 'E4c_M3_lekarske', 'E4d_M4_polno_les_vet', 'E4e_M5_spolocenske', 'E4f_M6_humanitne', 'E4g_M6_umenie'])
dict_keys(['2021 - 3'])
dict_keys(['crepc 2020', 'crepc 2021', 'crepc 2022', 'crepc 2022-2', 'CVTI_NORDIC', 'Nordic_CVTI 2020', 'Nordic_CVTI 2021', 'vystupy 2020', 'vystupy 2021', 'Hárok1'])
dict_keys(['T3 - výsk. zahr. grant. schémy', 'old', 'oblasti výskumu', 'VŠ', 'Odbory VaT'])


/home/fezjo/.local/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)
/home/fezjo/.local/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
/home/fezjo/.local/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)
/home/fezjo/.local/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [149]:
oblasti_mapping_sheet = prilohaA['E3_oblasti'].copy()

dictionary1 = oblasti_mapping_sheet.set_index(0)[2].to_dict()
dictionary2 = oblasti_mapping_sheet.set_index(1)[2].to_dict()
dictionary3 = {key: value for key, value in zip([
'PRÍRODNÉ VEDY',
'TECHNICKÉ VEDY', 
'LEKÁRSKE VEDY',
'PÔDOHOSPODÁRSKE VEDY',
'SPOLOČENSKÉ VEDY',
'HUMANITNÉ VEDY',
], [f'M{i}' for i in range(1,7)])}

oblasti_mapping = {**dictionary1, **dictionary2, **dictionary3}
oblasti_mapping

{91: 'M1',
 92: 'M1',
 120: 'M1',
 130: 'M1',
 240: 'M1',
 50: 'M2',
 100: 'M2',
 110: 'M2',
 140: 'M2',
 150: 'M2',
 160: 'M2',
 170: 'M2',
 180: 'M3',
 190: 'M4',
 200: 'M4',
 10: 'M5',
 60: 'M5',
 70: 'M5',
 80: 'M5',
 210: 'M5',
 220: 'M2',
 230: 'M5',
 20: 'M6',
 30: 'M6',
 40: 'M6',
 'fyzika': 'M1',
 'vedy o Zemi a vesmíre': 'M1',
 'chémia, chemická technológia a biotechnológie': 'M1',
 'vedy o živej prírode': 'M1',
 'matematika a štatistika': 'M1',
 'projektovanie, inžinierstvo, technológie a vodné hospodárstvo': 'M2',
 'environmentalistika a ekológia': 'M2',
 'metalurgické a montánne vedy': 'M2',
 'strojárstvo': 'M2',
 'elektrotechnika a elektroenergetika': 'M2',
 'informatické vedy, automatizácia a telekomunikácie': 'M2',
 'inžinierstvo a technológie': 'M2',
 'lekárske, farmaceutické a nelekárske zdravotnícke vedy': 'M3',
 'poľnohospodárske a lesnícke vedy': 'M4',
 'veterinárske vedy': 'M4',
 'pedagogické vedy': 'M5',
 'spoločenské a behaviorálne vedy': 'M5',
 'právo a medziná

In [232]:
vs_skratky = pd.read_csv('../data/vs_skratky.csv', index_col=0)
vs_skratky2cely_nazov = vs_skratky.reset_index().drop(columns=['cislo']).set_index('skratka_mesto')['VS_NAZOV'].to_dict()
vs_skratky2cely_nazov

{'UK Bratislava': 'Univerzita Komenského v Bratislave',
 'STU Bratislava': 'Slovenská technická univerzita v Bratislave',
 'EU Bratislava': 'Ekonomická univerzita v Bratislave',
 'SPU Nitra': 'Slovenská poľnohospodárska univerzita v Nitre',
 'TU Zvolen': 'Technická univerzita vo Zvolene',
 nan: 'Vysoká škola NEWTON, a.s. (NEWTON University)',
 'VŠMU Bratislava': 'Vysoká škola múzických umení v Bratislave',
 'UVLF Košice': 'Univerzita veterinárskeho lekárstva a farmácie v Košiciach',
 'TU Košice': 'Technická univerzita v Košiciach',
 'ŽU Žilina': 'Žilinská univerzita v Žiline',
 'UPJŠ Košice': 'Univerzita Pavla Jozefa Šafárika v Košiciach',
 'TVU Trnava': 'Trnavská univerzita v Trnave',
 'UMB Banská Bystrica': 'Univerzita Mateja Bela v Banskej Bystrici',
 'UKF Nitra': 'Univerzita Konštantína Filozofa v Nitre',
 'PU Prešov': 'Prešovská univerzita v Prešove',
 'TUAD Trenčín': 'Trenčianska univerzita Alexandra Dubčeka v Trenčíne',
 'UCM Trnava': 'Univerzita sv. Cyrila a Metoda v Trnave',
 

a) excelentná publikačná činnosť podľa bodu 4, celková váha 60%.

a) V oblastiach **M1**: prírodné vedy, **M2**: technické vedy, **M3**: lekárske vedy, **M4**: poľnohospodárske, lesnícke a veterinárske vedy sa zohľadňujú **váhou 100%** indexované publikácie (WoS, Scopus), teda kategórie **ADC, ADD, ADM, ADN**. Zoznam sa nachádza **v prílohe B**. Všetky výstupy sa **váhujú** podľa pracovísk tak, aby každý výstup bol zarátaný iba raz (súčet všetkých váh za daný **výstup je 1,00**).

b) V oblasti **M5**: spoločenské vedy sa zohľadňujú **váhou 75%** indexované publikácie **(WoS, Scopus)**, teda kategórie **ADC, ADD, ADM, ADN**. Tieto výstupy sa bonifikujú v súlade s metodikou rozpisu **(prvý kvartil JCR je zohľadnený váhou 6, druhý kvartil váhou 4, tretí kvartil váhou 1 a štvrtý kvartil váhou 0,5)**. Zoznam sa nachádza **v prílohe B**. **Váhou 25%** sa zohľadňujú monografie (**AAA, AAB**). Tieto výstupy sa **bonifikujú** tak, že vydavateľstvá zaradené v databáze Nordic List (https://kanalregister.hkdir.no/publiseringskanaler/Forside.action?request_locale=en) v kategórii 2 (prestížne vedecké vydavateľstvo) sú bonifikované váhou 6, vydavateľstvá zaradené v databáze Nordic List v kategórii 1 (vedecké vydavateľstvo) váhou 4, vydavateľstvá zaradené v databáze CVTI (zoznam zahraničných vydavateľstiev: http://cms.crepc.sk/Data/Sites/1/pdf/zoznam-vydavatelstva/zoznam-vydavatelstva-01-2020.pdf) sú zohľadnené váhou 1 a všetky ostatné monografie sú zohľadnené váhou 0,5). Zoznam aj so zaradením sa nachádza v prílohe C. Všetky výstupy sa váhujú podľa pracovísk tak, aby každý výstup bol zarátaný iba raz (súčet všetkých váh za daný výstup je 1,00).

c) V oblasti M6a: humanitné vedy sa zohľadňujú váhou 60% indexované publikácie (WoS, Scopus), teda kategórie ADC, ADD, ADM, ADN. Tieto výstupy sa bonifikujú v súlade s metodikou rozpisu (prvý kvartil JCR je zohľadnený váhou 6, druhý kvartil váhou 4, tretí kvartil váhou 1 a štvrtý kvartil váhou 0,5). Zoznam sa nachádza v prílohe B. Váhou 40% sa zohľadňujú monografie (AAA, AAB). Tieto výstupy sa bonifikujú tak, že vydavateľstvá zaradené v databáze Nordic List (https://kanalregister.hkdir.no/publiseringskanaler/Forside.action?request_locale=en) v kategórii 2 (prestížne vedecké vydavateľstvo) sú bonifikované váhou 6, vydavateľstvá zaradené v databáze Nordic List v kategórii 1 (vedecké vydavateľstvo) váhou 4, vydavateľstvá zaradené v databáze CVTI (zoznam zahraničných vydavateľstiev: http://cms.crepc.sk/Data/Sites/1/pdf/zoznam-vydavatelstva/zoznam-vydavatelstva-01-2020.pdf) sú zohľadnené váhou 1 a všetky ostatné monografie sú zohľadnené váhou 0,5). Zoznam aj so zaradením sa nachádza v prílohe C. Všetky výstupy sa váhujú podľa pracovísk tak, aby každý výstup bol zarátaný iba raz (súčet všetkých váh za daný výstup je 1,00).

d) v oblasti M6b umenie sa zohľadňujú váhou 100% umelecké výstupy v kategóriách ZZZ, ZZY, ZYZ, ZYY podľa váh platných pre rozpis dotácie 2022. Údaje sa nachádzajú v rozpise dotácie na rok 2022, hárok E4g_M6_umenie. Osobitne sa zohľadňujú oblasti performatívne umenie a vizuálne umenie.

In [151]:
# prilohy B, C
epc_codes_ADx = ['ADC', 'ADD', 'ADM', 'ADN']
epc_codes_AAx = ['AAA', 'AAB']
epc_codes_Zxx = ['ZZZ', 'ZZY', 'ZYZ', 'ZYY']

epc_codes_M1_M4 = epc_codes_ADx
epc_codes_M5 = epc_codes_ADx + epc_codes_AAx # TODO bonifikacia
epc_codes_M6a = epc_codes_ADx + epc_codes_AAx # TODO bonifikacia
epc_codes_M6b = epc_codes_Zxx # E4g_M6_umenie

publikacie_ADx = prilohaB['2021 - 3'].copy()
assert publikacie_ADx['EPC_KOD'].isin(epc_codes_ADx).all()

publikacie_AAx_2020 = prilohaC['vystupy 2020'].copy()
assert publikacie_AAx_2020['EPC_KOD'].isin(epc_codes_AAx).all()

publikacie_AAx_2021 = prilohaC['vystupy 2021'].copy()
assert publikacie_AAx_2021['EPC_KOD'].isin(epc_codes_AAx).all()

publikacie_AAx = pd.concat([publikacie_AAx_2020, publikacie_AAx_2021])

# TODO filter out only 2020-2021
casovy_ramec = [2020, 2021]

vysledne_pocty_fakult_v_oblastiach = {
    'M1': 24,
    'M2': 40,
    'M3': 11,
    'M4': 10,
    'M5': 70,
    'M6': 47,
}

In [152]:
def melt_sheet(sheet, relevant_columns, melt_columns, output_column):
    return (
        sheet.loc[:, relevant_columns + melt_columns]
            .melt(id_vars=relevant_columns, value_name=output_column)
            .drop(columns = ['variable'])
            .dropna(subset = [output_column])
    )

In [153]:
# inverzny dict
# vyfiltrovat iba riadky co maju spravny kod
# 

stlpce = ['VS_NAZOV', 'FAKULTA_NAZOV', 'Výskyt', 'Váha', 'JCR_bonus']
melt_stlpce = [f"OBLAST_VYSKUMU_KOD{i}" for i in ["", 2, 3, 4, 5]]
publikacie_ADx_melted = melt_sheet(publikacie_ADx, stlpce, melt_stlpce, 'oblast_kod')
publikacie_ADx_melted['oblast'] = publikacie_ADx_melted['oblast_kod'].map(oblasti_mapping)
publikacie_ADx_melted


,VS_NAZOV,FAKULTA_NAZOV,Výskyt,Váha,JCR_bonus,oblast_kod,oblast
0,Prešovská univerzita v Prešove,Gréckokatolícka teologická fakulta,1,1.000000,0.5,20.0,M6
1,Katolícka univerzita v Ružomberku,Teologická fakulta,1,1.000000,0.5,20.0,M6
2,Univerzita Konštantína Filozofa v Nitre,Filozofická fakulta,1,1.000000,0.5,20.0,M6
3,Katolícka univerzita v Ružomberku,Teologická fakulta,1,1.000000,0.5,20.0,M6
4,Univerzita Konštantína Filozofa v Nitre,Filozofická fakulta,3,0.333333,0.5,20.0,M6
...,...,...,...,...,...,...,...
74959,Technická univerzita v Košiciach,"Fakulta baníctva, ekológie, riadenia a geotech...",4,0.250000,4.0,170.0,M2
75460,Akadémia Policajného zboru v Bratislave,Pracoviská APZ,2,0.500000,0.5,80.0,M5
75461,Univerzita Mateja Bela v Banskej Bystrici,Fakulta politických vied a medzinárodných vzťahov,2,0.500000,0.5,80.0,M5
86523,Univerzita Konštantína Filozofa v Nitre,Fakulta prírodných vied a informatiky,2,0.500000,0.5,240.0,M1


In [154]:
index_columns = ['VS_NAZOV', 'FAKULTA_NAZOV']

def sumup_groupby(sheet, groupby_columns, sumup_column, output_column):
    return (
        sheet
            .groupby(groupby_columns, dropna = False)[sumup_column]
            .sum()
            .rename(output_column)
    )

# {
#     oblast: publikacie_ADx_melted[publikacie_ADx_melted['oblast'] == oblast]
#         .loc[:, index_columns]
#         .drop_duplicates()
#         .set_index(index_columns)
#     for oblast in set(oblasti_mapping.values())
# }

In [155]:
publikacie_ADx_sum = sumup_groupby(publikacie_ADx_melted, ['oblast'] + index_columns, 'Váha', 'sum_ADx')
publikacie_ADx_sum.sort_values(ascending=False)

oblast  VS_NAZOV                                             FAKULTA_NAZOV                              
M1      Univerzita Komenského v Bratislave                   Prírodovedecká fakulta UK                      942.595635
M3      Univerzita Komenského v Bratislave                   Lekárska fakulta UK                            764.305556
M1      Univerzita Komenského v Bratislave                   Fakulta matematiky, fyziky a informatiky UK    589.422222
        Univerzita Pavla Jozefa Šafárika v Košiciach         Prírodovedecká fakulta                         497.566667
M3      Univerzita Komenského v Bratislave                   Jesseniova lekárska fakulta UK                 459.194444
                                                                                                               ...    
M5      Univerzita Komenského v Bratislave                   Vedecký park UK                                  0.200000
M3      Prešovská univerzita v Prešove                       G

In [156]:
stlpce = ['VS_NAZOV', 'FAKULTA_NAZOV', 'kategoria_metodika']
melt_stlpce = [f"OBLAST_VYSKUMU_KOD{i}" for i in ["", 2]]
publikacie_AAx_melted = melt_sheet(publikacie_AAx, stlpce, melt_stlpce, 'oblast_kod')
publikacie_AAx_melted['oblast'] = publikacie_AAx_melted['oblast_kod'].map(oblasti_mapping)
publikacie_AAx_melted.query('VS_NAZOV == "Ekonomická univerzita v Bratislave" and FAKULTA_NAZOV == "Obchodná fakulta"')

,VS_NAZOV,FAKULTA_NAZOV,kategoria_metodika,oblast_kod,oblast
1,Ekonomická univerzita v Bratislave,Obchodná fakulta,0.5,80.0,M5
2,Ekonomická univerzita v Bratislave,Obchodná fakulta,0.5,80.0,M5
64,Ekonomická univerzita v Bratislave,Obchodná fakulta,0.5,80.0,M5
65,Ekonomická univerzita v Bratislave,Obchodná fakulta,0.5,80.0,M5
67,Ekonomická univerzita v Bratislave,Obchodná fakulta,0.5,80.0,M5
135,Ekonomická univerzita v Bratislave,Obchodná fakulta,0.5,80.0,M5
170,Ekonomická univerzita v Bratislave,Obchodná fakulta,0.5,80.0,M5
171,Ekonomická univerzita v Bratislave,Obchodná fakulta,0.5,80.0,M5
172,Ekonomická univerzita v Bratislave,Obchodná fakulta,0.5,80.0,M5
173,Ekonomická univerzita v Bratislave,Obchodná fakulta,0.5,80.0,M5


In [157]:
publikacie_AAx_sum = sumup_groupby(publikacie_AAx_melted, ['oblast'] + index_columns, 'kategoria_metodika', 'sum_AAx')
publikacie_AAx_sum.sort_values(ascending=False)

oblast  VS_NAZOV                                   FAKULTA_NAZOV                                 
M5      Univerzita Komenského v Bratislave         Právnická fakulta UK                              74.0
        Univerzita Mateja Bela v Banskej Bystrici  Právnická fakulta                                 49.0
        Ekonomická univerzita v Bratislave         Obchodná fakulta                                  32.5
M6      Prešovská univerzita v Prešove             Filozofická fakulta                               31.5
M5      Trnavská univerzita v Trnave               Právnická fakulta                                 31.0
                                                                                                     ... 
M1      Žilinská univerzita v Žiline               Fakulta prevádzky a ekonomiky dopravy a spojov     0.5
M5      Univerzita Mateja Bela v Banskej Bystrici  Fakulta humanitných vied                           0.5
M6      Technická univerzita v Košiciach           Eko

5. Excelentný výkon pri získavaní výskumných grantov sa určuje nasledovne: pre všetky oblasti okrem M6b (umenie) sa zohľadňuje objem grantových prostriedkov, získaných súťažným spôsobom v kategórii zahraničné výskumné granty. Zoznam sa nachádza v prílohe D. Zahraničné výskumné granty sa priraďujú iba tým pracoviskám, v ktorých vysoké školy uviedli príslušných zamestnancov v danej oblasti.



In [202]:
granty_orig = prilohaD['T3 - výsk. zahr. grant. schémy'].copy()
# granty_orig = granty_orig[granty_orig['Rok'].isin(casovy_ramec)]

vyhra_key = '''Výška finančných prostriedkov v kategórii BV prijatých vysokou školou na jej účet v období od 1.1. do 31.12.2021
(uviesť v eurách v celých jednotkách)'''
granty_orig.rename(columns = {
    vyhra_key: 'Výhra',
    'Vysoká škola': 'VS_NAZOV',
    'Názov fakulty': 'FAKULTA_NAZOV',
    'Identifikačné číslo projektu podľa zmluvy': 'ID',
    }, inplace = True)

relevant_columns = index_columns + ['SKUPINA ODBOROV VEDY A TECHNIKY', 'Rok', 'ID', 'Výhra']
granty_orig = granty_orig.loc[:, relevant_columns].dropna(subset = ['Výhra'])
granty_orig['FAKULTA_NAZOV'] = granty_orig['FAKULTA_NAZOV'].fillna('<neuvedené>')

granty_orig['oblast'] = granty_orig['SKUPINA ODBOROV VEDY A TECHNIKY'].map(oblasti_mapping)
granty_orig

,VS_NAZOV,FAKULTA_NAZOV,SKUPINA ODBOROV VEDY A TECHNIKY,Rok,ID,Výhra,oblast
0,STU Bratislava,Fakulta chemickej a potravinárskej technológie...,TECHNICKÉ VEDY,2021,122,22025.58,M2
1,STU Bratislava,Fakulta chemickej a potravinárskej technológie...,PRÍRODNÉ VEDY,2020,122,8045.52,M1
2,ŽU Žilina,Fakulta prevádzky a ekonomiky dopravy a spojov ŽU,SPOLOČENSKÉ VEDY,2021,17040,11324.00,M5
3,TU Košice,Ekonomická fakulta TUKE,SPOLOČENSKÉ VEDY,2020,17123,9635.00,M5
4,TU Košice,"Fakulta baníctva, ekológie, riadenia a geotech...",TECHNICKÉ VEDY,2021,18259,84860.34,M2
...,...,...,...,...,...,...,...
487,STU Bratislava,Materiálovotechnologická fakulta STU v Trnave,TECHNICKÉ VEDY,2020,NaN,3063.45,M2
488,STU Bratislava,Materiálovotechnologická fakulta STU v Trnave,TECHNICKÉ VEDY,2020,NaN,33138.00,M2
489,STU Bratislava,Ústav manažmentu STU,SPOLOČENSKÉ VEDY,2020,NaN,40438.79,M5
490,STU Bratislava,Ústav manažmentu STU,SPOLOČENSKÉ VEDY,2020,NaN,110344.48,M5


In [207]:
granty_orig.loc[granty_orig['ID'] == 'H2020 739566', 'FAKULTA_NAZOV'] = 'FunGlass - Centrum pre funkčné a povrchovo funkcionalizované sklá'


In [235]:
from collections import defaultdict
import string

translate_table = str.maketrans('', '', string.punctuation)
tokenize = lambda s: set(s.translate(translate_table).lower().split())

stare_nazvy = list(set(granty_orig.set_index(['VS_NAZOV', 'FAKULTA_NAZOV']).index))
fakulty = defaultdict(lambda: [[], []])

for vs, fakulta in stare_nazvy:
    vs_nazov = vs_skratky2cely_nazov[vs]
    fakulty[vs_nazov][0].append((vs, fakulta))

nove_nazvy = list(set(pocty_zamestnancov.index))
for vs, fakulta in nove_nazvy:
    fakulty[vs][1].append(fakulta)

granty2publikacie_naming_mapping = {}
for vs_nove in fakulty:
    stare, nove = fakulty[vs_nove]
    if not nove:
        print('UNKNOWN VS', vs_nove, stare)
        nove = ['unknown']
    for vs_stara, fakulta_stara in stare:
        matching_to = fakulta_stara.translate(translate_table).split()
        if len(matching_to[-1]) <= 4 and matching_to[-1].isupper():
            matching_to.pop()
        matching_to = " ".join(matching_to)

        podobnosti = []
        for fakulta_nova in nove:
            podobnosti.append(len(tokenize(matching_to) & tokenize(fakulta_nova.lower())))
        best_i = podobnosti.index(max(podobnosti))
        fakulta_nova = nove[best_i]
        dlzka1 = len(tokenize(matching_to))
        dlzka2 = len(tokenize(nove[best_i]))
        if podobnosti[best_i] != dlzka2 and podobnosti[best_i] < dlzka1 * 0.75:
            print(fakulta_stara, best_i, dlzka1, dlzka2, podobnosti[best_i], fakulta_nova)
            fakulta_nova = f'unknown[{fakulta_stara}]'
        granty2publikacie_naming_mapping[(vs_stara, fakulta_stara)] = (vs_nove, fakulta_nova)

granty2publikacie_naming_mapping

Rektorát ŽU_ERAdiate+ 0 2 4 0 Univerzitný vedecký park UNIZA
Rektorát ŽU/ERAdiate+ 0 2 4 0 Univerzitný vedecký park UNIZA
Ústav znaleckého výskumu a vzdelávania ŽU 2 5 5 1 Fakulta elektrotechniky a informačných technológií
Rektorát UK 0 1 3 0 Lekárska fakulta UK
<neuvedené> 0 1 3 0 Fakulta verejnej správy
Fakulta zdravotníctva PU 0 2 2 1 Pedagogická fakulta
Rektorát 0 1 2 0 Pedagogická fakulta
vyberte, prosím 0 2 2 0 Inštitút manažmentu
<neuvedené> 0 1 2 0 Inštitút manažmentu
vyberte, prosím 0 2 6 0 Materiálovotechnologická fakulta so sídlom v Trnave
<neuvedené> 0 1 6 0 Materiálovotechnologická fakulta so sídlom v Trnave
UVLF Košice 0 2 8 0 Pracoviská hygieny potravín, chovu zvierat a životného prostredia
Univerzita veterinárskeho lekárstva v Košiciach 0 5 8 0 Pracoviská hygieny potravín, chovu zvierat a životného prostredia
Rektorát 0 1 3 0 Centrum jazykov SPU
Ekonomická fakulta UJS 0 2 4 1 Fakulta ekonómie a informatiky


{('ŽU Žilina', 'Stavebná fakulta ŽU'): ('Žilinská univerzita v Žiline',
  'Stavebná fakulta'),
 ('ŽU Žilina',
  'Fakulta bezpečnostného inžinierstva ŽU'): ('Žilinská univerzita v Žiline', 'Fakulta bezpečnostného inžinierstva'),
 ('ŽU Žilina', 'Rektorát ŽU_ERAdiate+'): ('Žilinská univerzita v Žiline',
  'unknown[Rektorát ŽU_ERAdiate+]'),
 ('ŽU Žilina', 'Rektorát ŽU/ERAdiate+'): ('Žilinská univerzita v Žiline',
  'unknown[Rektorát ŽU/ERAdiate+]'),
 ('ŽU Žilina', 'Výskumné centrum ŽU'): ('Žilinská univerzita v Žiline',
  'Výskumné centrum UNIZA'),
 ('ŽU Žilina',
  'Fakulta prevádzky a ekonomiky dopravy a spojov ŽU'): ('Žilinská univerzita v Žiline', 'Fakulta prevádzky a ekonomiky dopravy a spojov'),
 ('ŽU Žilina',
  'Ústav znaleckého výskumu a vzdelávania ŽU'): ('Žilinská univerzita v Žiline', 'unknown[Ústav znaleckého výskumu a vzdelávania ŽU]'),
 ('ŽU Žilina',
  'Fakulta elektrotechniky a informačných technológií ŽU'): ('Žilinská univerzita v Žiline', 'Fakulta elektrotechniky a informač

In [237]:
granty = granty_orig.copy().set_index(['VS_NAZOV', 'FAKULTA_NAZOV'])
granty.index = granty.index.map(granty2publikacie_naming_mapping)
granty = granty.reset_index()
granty_unknown = granty[granty['FAKULTA_NAZOV'].str.startswith('unknown')].groupby(['VS_NAZOV', 'FAKULTA_NAZOV']).sum('Výhra')
total_granty, total_granty_unknown = granty['Výhra'].sum(), granty_unknown['Výhra'].sum()
print(f'Total: {total_granty:.0f}€, Unknown: {total_granty_unknown:.0f}€, {total_granty_unknown / total_granty:.1%}')
display(granty_unknown.sort_values('Výhra', ascending=False))
granty = granty.set_index(['oblast', 'VS_NAZOV', 'FAKULTA_NAZOV'])
granty.sort_values('Výhra', ascending=False)

Total: 24135069€, Unknown: 2070715€, 8.6%


Rok  \
VS_NAZOV                                           FAKULTA_NAZOV                                              
Slovenská poľnohospodárska univerzita v Nitre      unknown[Rektorát]                                   2021   
Žilinská univerzita v Žiline                       unknown[Rektorát ŽU/ERAdiate+]                      8080   
Univerzita Pavla Jozefa Šafárika v Košiciach       unknown[<neuvedené>]                                2020   
Prešovská univerzita v Prešove                     unknown[Rektorát]                                   4042   
Univerzita Komenského v Bratislave                 unknown[Rektorát UK]                                2021   
Žilinská univerzita v Žiline                       unknown[Rektorát ŽU_ERAdiate+]                      4042   
Slovenská technická univerzita v Bratislave        unknown[<neuvedené>]                                8080   
                                                   unknown[vyberte, prosím]                            8084   
Univerzita veterinárskeho lekárstva a farmácie ... unknown[Univerzita veterinárskeho lekárstva v K...  4042   
Žilinská univerzita v Žiline                       unknown[Ústav znaleckého výskumu a vzdelávania ŽU]  4040   
Univerzita sv. Cyrila a Metoda v Trnave            unknown[<neuvedené>]                                2020   
Prešovská univerzita v Prešove                     unknown[Fakulta zdravotníctva PU]                   2021   
Univerzita J. Selyeho                              unknown[Ekonomická fakulta UJS]                     2021   
Univerzita veterinárskeho lekárstva a farmácie ... unknown[UVLF Košice]                                2020   
Univerzita sv. Cyrila a Metoda v Trnave            unknown[vyberte, prosím]                            4042   

                                                                                                           Výhra  
VS_NAZOV                                           FAKULTA_NAZOV                                                  
Slovenská poľnohospodárska univerzita v Nitre      unknown[Rektorát]                                   563890.62  
Žilinská univerzita v Žiline                       unknown[Rektorát ŽU/ERAdiate+]                      324419.00  
Univerzita Pavla Jozefa Šafárika v Košiciach       unknown[<neuvedené>]                                323296.88  
Prešovská univerzita v Prešove                     unknown[Rektorát]                                   210444.34  
Univerzita Komenského v Bratislave                 unknown[Rektorát UK]                                180000.00  
Žilinská univerzita v Žiline                       unknown[Rektorát ŽU_ERAdiate+]                      136035.00  
Slovenská technická univerzita v Bratislave        unknown[<neuvedené>]                                109323.25  
                                                   unknown[vyberte, prosím]                             96155.76  
Univerzita veterinárskeho lekárstva a farmácie ... unknown[Univerzita veterinárskeho lekárstva v K...   34037.00  
Žilinská univerzita v Žiline                       unknown[Ústav znaleckého výskumu a vzdelávania ŽU]   30081.00  
Univerzita sv. Cyrila a Metoda v Trnave            unknown[<neuvedené>]                                 22557.00  
Prešovská univerzita v Prešove                     unknown[Fakulta zdravotníctva PU]                    21208.00  
Univerzita J. Selyeho                              unknown[Ekonomická fakulta UJS]                      11768.00  
Univerzita veterinárskeho lekárstva a farmácie ... unknown[UVLF Košice]                                  7499.00  
Univerzita sv. Cyrila a Metoda v Trnave            unknown[vyberte, prosím]                                 0.00

SKUPINA ODBOROV VEDY A TECHNIKY  \
oblast VS_NAZOV                                           FAKULTA_NAZOV                                                                        
M1     Trenčianska univerzita Alexandra Dubčeka v Tren... FunGlass - Centrum pre funkčné a povrchovo funk...                   PRÍRODNÉ VEDY   
M2     Trenčianska univerzita Alexandra Dubčeka v Tren... FunGlass - Centrum pre funkčné a povrchovo funk...                  TECHNICKÉ VEDY   
M3     Trnavská univerzita v Trnave                       Fakulta zdravotníctva a sociálnej práce                              LEKÁRSKE VEDY   
M5     Slovenská poľnohospodárska univerzita v Nitre      unknown[Rektorát]                                                 SPOLOČENSKÉ VEDY   
       Technická univerzita v Košiciach                   Ekonomická fakulta                                                SPOLOČENSKÉ VEDY   
...                                                                                                                                      ...   
       Univerzita Mateja Bela v Banskej Bystrici          Filozofická fakulta                                               SPOLOČENSKÉ VEDY   
M2     Slovenská technická univerzita v Bratislave        Materiálovotechnologická fakulta so sídlom v Tr...                  TECHNICKÉ VEDY   
M3     Univerzita sv. Cyrila a Metoda v Trnave            unknown[vyberte, prosím]                                             LEKÁRSKE VEDY   
M4     Univerzita sv. Cyrila a Metoda v Trnave            Fakulta prírodných vied                                       PÔDOHOSPODÁRSKE VEDY   
M5     Ekonomická univerzita v Bratislave                 Obchodná fakulta                                                  SPOLOČENSKÉ VEDY   

                                                                                                               Rok  \
oblast VS_NAZOV                                           FAKULTA_NAZOV                                              
M1     Trenčianska univerzita Alexandra Dubčeka v Tren... FunGlass - Centrum pre funkčné a povrchovo funk...  2020   
M2     Trenčianska univerzita Alexandra Dubčeka v Tren... FunGlass - Centrum pre funkčné a povrchovo funk...  2021   
M3     Trnavská univerzita v Trnave                       Fakulta zdravotníctva a sociálnej práce             2021   
M5     Slovenská poľnohospodárska univerzita v Nitre      unknown[Rektorát]                                   2021   
       Technická univerzita v Košiciach                   Ekonomická fakulta                                  2021   
...                                                                                                            ...   
       Univerzita Mateja Bela v Banskej Bystrici          Filozofická fakulta                                 2021   
M2     Slovenská technická univerzita v Bratislave        Materiálovotechnologická fakulta so sídlom v Tr...  2021   
M3     Univerzita sv. Cyrila a Metoda v Trnave            unknown[vyberte, prosím]                            2021   
M4     Univerzita sv. Cyrila a Metoda v Trnave            Fakulta prírodných vied                             2021   
M5     Ekonomická univerzita v Bratislave                 Obchodná fakulta                                    2021   

                                                                                                                                                           ID  \
oblast VS_NAZOV                                           FAKULTA_NAZOV                                                                                         
M1     Trenčianska univerzita Alexandra Dubčeka v Tren... FunGlass - Centrum pre funkčné a povrchovo funk...                                     H2020 739566   
M2     Trenčianska univerzita Alexandra Dubčeka v Tren... FunGlass - Centrum pre funkčné a povrchovo funk...                                     H2020 739566   
M3     Trnavská univerzita v Trnave                      

In [161]:
# # # TODO fakulty_v_oblasti do novej premennej a merge so starou
# fakulty_v_oblasti = {
#     oblast: granty[granty['oblast'] == oblast]
#         .loc[:, index_columns]
#         .drop_duplicates()
#         .set_index(index_columns)
#     for oblast in set(oblasti_mapping.values())
# }

# for value in fakulty_v_oblasti.values():
#     value['sum_granty'] = 0.0

# for i, riadok in granty.iterrows():
#     pracovisko = tuple(riadok[key] for key in index_columns)
#     fakulty_v_oblasti[riadok['oblast']].loc[pracovisko]['sum_granty'] += riadok['Výhra']


# fakulty_v_oblasti['M1'].sort_values(by='sum_granty', ascending=False)

In [238]:
granty_sum = sumup_groupby(granty, ['oblast'] + index_columns, 'Výhra', 'sum_granty')
granty_sum.loc['M1'].sort_values(ascending=False)

granty_sum = granty_sum.reset_index()
granty_sum = granty_sum.set_index(['oblast', 'VS_NAZOV', 'FAKULTA_NAZOV'])['sum_granty']
granty_sum

oblast  VS_NAZOV                                       FAKULTA_NAZOV                                      
M1      Prešovská univerzita v Prešove                 Fakulta humanitných a prírodných vied                   20000.00
        Slovenská poľnohospodárska univerzita v Nitre  Fakulta biotechnológie a potravinárstva                 33400.00
        Slovenská technická univerzita v Bratislave    Fakulta chemickej a potravinárskej technológie          92845.71
                                                       Fakulta elektrotechniky a informatiky                  271309.02
                                                       Materiálovotechnologická fakulta so sídlom v Trnave     15000.00
                                                                                                                ...    
M6      Trnavská univerzita v Trnave                   Pedagogická fakulta                                      2461.00
        Univerzita Komenského v Bratislave           

In [239]:
pocty_zamestnancov = prilohaA['E2_zamestnanci_2021'].copy()
pocty_zamestnancov.columns = pocty_zamestnancov.iloc[0].combine_first(pocty_zamestnancov.iloc[1])
pocty_zamestnancov = pocty_zamestnancov[2:]
pocty_zamestnancov = (
    pocty_zamestnancov[pocty_zamestnancov['kod skoly'].notna()]
    .drop(columns=['kod skoly'])
    .rename(columns={'FAKULTA_NAZOV / sucasti': 'FAKULTA_NAZOV'})
    .set_index(index_columns)
    .fillna(0)
    .astype(int)
    .groupby(oblasti_mapping, axis=1)
    .sum()
)
pocty_zamestnancov

0                                                                  M1  M2  \
VS_NAZOV                           FAKULTA_NAZOV                            
Univerzita Komenského v Bratislave Právnická fakulta UK             0   0   
                                   Pedagogická fakulta UK           0   0   
                                   Filozofická fakulta UK           0   0   
                                   Jesseniova lekárska fakulta UK   0   0   
                                   Fakulta managementu UK           0   0   
...                                                                ..  ..   
Katolícka univerzita v Ružomberku  Filozofická fakulta              0   0   
                                   Fakulta zdravotníctva            0   0   
Univerzita J. Selyeho              Pedagogická fakulta              0   0   
                                   Fakulta ekonómie a informatiky  10  14   
                                   Reformovaná teologická fakulta   0   0   

0                                                                   M3  M4  \
VS_NAZOV                           FAKULTA_NAZOV                             
Univerzita Komenského v Bratislave Právnická fakulta UK              0   0   
                                   Pedagogická fakulta UK            0   0   
                                   Filozofická fakulta UK            0   0   
                                   Jesseniova lekárska fakulta UK  224   0   
                                   Fakulta managementu UK            1   0   
...                                                                ...  ..   
Katolícka univerzita v Ružomberku  Filozofická fakulta               0   0   
                                   Fakulta zdravotníctva            41   0   
Univerzita J. Selyeho              Pedagogická fakulta               0   0   
                                   Fakulta ekonómie a informatiky    0   0   
                                   Reformovaná teologická fakulta    0   0   

0                                                                   M5   M6  
VS_NAZOV                           FAKULTA_NAZOV                             
Univerzita Komenského v Bratislave Právnická fakulta UK            137    0  
                                   Pedagogická fakulta UK          131    9  
                                   Filozofická fakulta UK           71  183  
                                   Jesseniova lekárska fakulta UK    0    6  
                                   Fakulta managementu UK           74    3  
...                                                                ...  ...  
Katolícka univerzita v Ružomberku  Filozofická fakulta              37    7  
                                   Fakulta zdravotníctva             0    0  
Univerzita J. Selyeho              Pedagogická fakulta              53    9  
                                   Fakulta ekonómie a informatiky   18    0  
                                   Reformovaná teologická fakulta    0   11  

[128 rows x 6 columns]

In [241]:
uni_adx = set(map(lambda x: x[1:], publikacie_ADx_sum.index))
uni_aax = set(map(lambda x: x[1:], publikacie_AAx_sum.index))
uni_granty = set(map(lambda x: x[1:], granty_sum.index))
print(list(map(len, [uni_adx, uni_aax, uni_granty])))
print(len(uni_adx & uni_aax))
print(len(uni_adx & uni_granty))
print(len(uni_aax & uni_granty))
print(len((uni_aax | uni_adx) & uni_granty))
display(uni_adx | uni_aax)
display(uni_granty - uni_adx - uni_aax)

[158, 128, 91]
122
72
68
74


{('Akadémia Policajného zboru v Bratislave', 'Pracoviská APZ'),
 ('Akadémia Policajného zboru v Bratislave', 'Rektorát'),
 ('Akadémia ozbrojených síl generála Milana Rastislava Štefánika',
  'Pracoviská AOS'),
 ('Akadémia umení', 'Fakulta dramatických umení'),
 ('Akadémia umení', 'Fakulta múzických umení'),
 ('Ekonomická univerzita v Bratislave', 'Celouniverzitné pracovisko EUBA'),
 ('Ekonomická univerzita v Bratislave', 'Fakulta aplikovaných jazykov'),
 ('Ekonomická univerzita v Bratislave', 'Fakulta hospodárskej informatiky'),
 ('Ekonomická univerzita v Bratislave', 'Fakulta medzinárodných vzťahov'),
 ('Ekonomická univerzita v Bratislave', 'Fakulta podnikového manažmentu'),
 ('Ekonomická univerzita v Bratislave', 'Národohospodárska fakulta'),
 ('Ekonomická univerzita v Bratislave', 'Obchodná fakulta'),
 ('Ekonomická univerzita v Bratislave',
  'Podnikovohospodárska fakulta v Košiciach'),
 ('Ekonomická univerzita v Bratislave', nan),
 ('Katolícka univerzita v Ružomberku', 'Fakulta zdr

{('Prešovská univerzita v Prešove', 'unknown[Fakulta zdravotníctva PU]'),
 ('Prešovská univerzita v Prešove', 'unknown[Rektorát]'),
 ('Slovenská poľnohospodárska univerzita v Nitre', 'unknown[Rektorát]'),
 ('Slovenská technická univerzita v Bratislave', 'unknown[<neuvedené>]'),
 ('Slovenská technická univerzita v Bratislave', 'unknown[vyberte, prosím]'),
 ('Univerzita J. Selyeho', 'unknown[Ekonomická fakulta UJS]'),
 ('Univerzita Komenského v Bratislave', 'unknown[Rektorát UK]'),
 ('Univerzita Pavla Jozefa Šafárika v Košiciach', 'Lekárska fakulta, Košice'),
 ('Univerzita Pavla Jozefa Šafárika v Košiciach', 'unknown[<neuvedené>]'),
 ('Univerzita sv. Cyrila a Metoda v Trnave', 'unknown[<neuvedené>]'),
 ('Univerzita sv. Cyrila a Metoda v Trnave', 'unknown[vyberte, prosím]'),
 ('Univerzita veterinárskeho lekárstva a farmácie v Košiciach',
  'unknown[UVLF Košice]'),
 ('Univerzita veterinárskeho lekárstva a farmácie v Košiciach',
  'unknown[Univerzita veterinárskeho lekárstva v Košiciach]'),

In [165]:
pracoviska_excelentnost = pd.concat([
    publikacie_ADx_sum,
    publikacie_AAx_sum,
    granty_sum,
], axis=1, join='outer')
pracoviska_excelentnost.loc[['M1','M2','M3','M4'], 'publikacna_excelentnost'] = pracoviska_excelentnost.loc[['M1','M2','M3','M4']]['sum_ADx']
pracoviska_excelentnost.loc[['M5'], 'publikacna_excelentnost'] = pracoviska_excelentnost.loc[['M5']].apply(lambda x: x['sum_ADx'] * 0.75 + x['sum_AAx'] * 0.25, axis=1)
pracoviska_excelentnost.loc[['M6'], 'publikacna_excelentnost'] = pracoviska_excelentnost.loc[['M6']].apply(lambda x: x['sum_ADx'] * 0.60 + x['sum_AAx'] * 0.40, axis=1)
pracoviska_excelentnost.sort_values(by='publikacna_excelentnost', ascending=False)

ValueError: cannot handle a non-unique multi-index!

In [ ]:
for m in ['M4']: #pocty_zamestnancov.columns:
    # print(m)
    relevant_high = pocty_zamestnancov[m][pocty_zamestnancov[m] >= 5]
    relevant_low = pocty_zamestnancov[m][pocty_zamestnancov[m] >= 1][pocty_zamestnancov[m] < 5]
    relevant_zero = pocty_zamestnancov[m][pocty_zamestnancov[m] == 0]
    # print(pracoviska_excelentnost.loc[m])
    relevant_excelent = pracoviska_excelentnost.loc[m]
    relevant_excelent = relevant_excelent[relevant_excelent['sum_ADx'] + relevant_excelent['sum_AAx'] > 0]
    # print(relevant)
    print(f'm={m}, 0==x={len(relevant_zero)} 0<x<5={len(relevant_low)}, 5<=x={len(relevant_high)}, pub|grant={len(relevant_excelent)}, excel={vysledne_pocty_fakult_v_oblastiach[m]}')
    # print(relevant_high)
    # print(relevant_excelent)
    # print()

vysledne_pocty_fakult_v_oblastiach

m=M4, 0==x=116 0<x<5=1, 5<=x=11, pub|grant=59, excel=10


{'M1': 24, 'M2': 40, 'M3': 11, 'M4': 10, 'M5': 70, 'M6': 47}

In [ ]:
import subprocess

# export grant data to csv
# run r script
# import resulting csv containing predicted values
zamestnanci_publikacie = (
    pocty_zamestnancov
    .melt(ignore_index=False, var_name='oblast', value_name='pocty_zamestnancov')
    .reset_index()
    .set_index(['oblast', 'VS_NAZOV', 'FAKULTA_NAZOV'])
    .query('pocty_zamestnancov >= 5')
    .join(pracoviska_excelentnost[['publikacna_excelentnost', 'sum_granty']])
    # .fillna(0)
)
for oblast in sorted(set(oblasti_mapping.values())):
    print(oblast)
    print(zamestnanci_publikacie.loc[oblast])
    data = zamestnanci_publikacie.loc[oblast].reset_index()
    data.to_csv('data.csv', index=False)
    aaa

    subprocess.run(['Rscript', 'predict.R'])

    predicted = pd.read_csv('predicted.csv')
    pracoviska_excelentnost.loc[[oblast], 'publikacne_predicted'] = predicted

M1
                                                                                                       pocty_zamestnancov  \
VS_NAZOV                                           FAKULTA_NAZOV                                                            
Univerzita Komenského v Bratislave                 Prírodovedecká fakulta UK                                          296   
                                                   Fakulta matematiky, fyziky a informatiky UK                        174   
                                                   Lekárska fakulta UK                                                 33   
                                                   Vedecký park UK                                                     13   
Slovenská technická univerzita v Bratislave        Materiálovotechnologická fakulta so sídlom v Tr...                   9   
                                                   Stavebná fakulta STU v Bratislave                                   46 

NameError: name 'aaa' is not defined

## TODO

- [-] publikacie M6b Zxx
- [x] extrahovat pocty zamestnancov na pracoviskach
- [x] vypisat pracoviska v oblastiach ktore maju aspon 5 zamestancov
- [ ] namapovat nazvy skol v grantoch na publikacie
- [ ] porovnat publikacnu excelentnost s referencnymi datami
- [ ] publikacie ceil(val * 2)
- [ ] vyhodnotit regresne modely
- [ ] vypocitat rezidua
- [ ] standardizovat rezidua do z-skore
- [ ] prevahovat z-skore
- [ ] urcit horny kvartil z kazdej oblasti
- [ ] porovnat vypocitane z-skore s referencnymi vysledkami
- [ ] odhadnut vyhru
- [ ] spriemerovat s minulorocnou vyhrou
- [ ] preskalovat priemernu vyhru
- [ ] overit AAx duplicitu medzi roznymi pracoviskami
- [ ] ...
- [ ] vygenerovat nechutnu tabulku